<a href="https://colab.research.google.com/github/RaianaSeixas/0_paper_one_revisited/blob/main/CEC14/lshade/LSHADE_LPSR_F1_180_4_cec2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
############################### DADOS DE ENTRADA 
 
RUNS=50 #NÚMERO DE RODADAS
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
ITE=1000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100
mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
# fator = 1.0/3.0; 
fator = 1.0 # just LSHADE...
mut2=0.5;crossp2=0.5;mut3=0.75;crossp3=0.25;mut4=0.25;crossp4=0.25;
mut_best  =0.5 ; crossp_best =0.5
maxNP = 180
minNP = 4
maxFES =300000
FIX = 100 # Number of adjusts random walk
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]
MFi=0.5
MCRi=0.5

In [2]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz
import Go2Ann
import randomWALK
import annDE_FES_new
import classicDE_FES_new
import stepsWALK_new
import LShade_new
from pandas import read_csv

Cloning into 'optim_raianars_Insight'...
remote: Enumerating objects: 893, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 893 (delta 165), reused 124 (delta 82), pack-reused 641
Receiving objects: 100% (893/893), 453.00 KiB | 4.53 MiB/s, done.
Resolving deltas: 100% (555/555), done.
/content/optim_raianars_Insight


In [3]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

Cloning into 'Raianars_paper_One'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 277 (delta 83), reused 142 (delta 82), pack-reused 129
Receiving objects: 100% (277/277), 667.30 KiB | 13.35 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/optim_raianars_Insight/Raianars_paper_One


In [4]:
#import Go2Ann
#import de_soma_Insight
#import de_bounds_matriz
#import Function
import Go2MinMax
from AvaliaX import AvaliaX
import numpy as np
import scipy.stats
import random
from GeraIndices import GeraIndices
import numpy as np
import time
import matplotlib.pyplot as plt
from random import randint
from scipy import optimize
import pandas as pd
from sklearn.metrics import confusion_matrix

In [5]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz

Cloning into 'optim_raianars_Insight'...
remote: Enumerating objects: 893, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 893 (delta 165), reused 124 (delta 82), pack-reused 641
Receiving objects: 100% (893/893), 453.00 KiB | 8.71 MiB/s, done.
Resolving deltas: 100% (555/555), done.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight


In [6]:
#!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
#%cd Raianars_paper_One

!git clone https://github.com/ucfilho/Raianars_Routines #clonar do Github
%cd Raianars_Routines

Cloning into 'Raianars_Routines'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 77 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (77/77), done.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines


In [7]:
!git clone https://github.com/ucfilho/functions_cec2014_data
%cd functions_cec2014_data
import F1_to_F27_cec2014_zero_to_hero
import cec2014_names
import cec2014_shift_rotate

Cloning into 'functions_cec2014_data'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 300 (delta 182), reused 290 (delta 178), pack-reused 3
Receiving objects: 100% (300/300), 466.45 KiB | 7.65 MiB/s, done.
Resolving deltas: 100% (182/182), done.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data


In [8]:
function_select = 'F1'
Function = F1_to_F27_cec2014_zero_to_hero.F1
Shift_Rotate= cec2014_shift_rotate.F1
problem_size = 30 # possibles 2, 10, 20, 30, 50, 100

In [9]:
dim = problem_size 

In [10]:
f_matrix = "M_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
shift_data_file="shift_data_"+function_select[1:]+".txt"
F_bias = cec2014_names.BiasValue()[function_select]
type_fun = cec2014_names.TypeFunctions()[function_select]
Function_CEC2014 =function_select + "_" + cec2014_names.NamesFunctions()[function_select]+"_D_"+str(problem_size)

In [11]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# !gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
# https://drive.google.com/file/d/1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz/view?usp=sharing
!gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
To: /content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/cec2014.zip
100% 36.1M/36.1M [00:00<00:00, 226MB/s]


In [13]:
import zipfile
file_name = zipfile.ZipFile('cec2014.zip', 'r')
file_name.extractall()

In [14]:
def load_shift_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values.reshape((-1))

def load_matrix_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values

In [15]:
shift_data_a = load_shift_data__(shift_data_file)[:problem_size]
matrix_a = load_matrix_data__(f_matrix)
if (type_fun == 'hybrid'):
  f_shuffle = "shuffle_data_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
  shuffle = (load_shift_data__(f_shuffle)[:problem_size] - np.ones(problem_size)).astype(int)
else:
  shuffle = matrix_a

In [16]:
SUPPORT_DIMENSION = [2, 10, 20, 30, 50, 100]
SUPPORT_DIMENSION_2 = [10, 20, 30, 50, 100]

#def Fun(solution,shift_data=shift_data_a, matrix=matrix_a, shuffle):#, bias=F_bias):
def CEC2014(solution, shift_data ,  matrix, shuffle):#, bias=F_bias):
  if(type_fun == 'shift_rotate'):
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = Function(z) + F_bias
  elif(type_fun =='hybrid'): # F17 to F22
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = z + F_bias
 
  return result

In [17]:
 shift_data = shift_data_a
 matrix = matrix_a
 shuffle = shuffle
 def Hybrid(solution):
   resp = CEC2014(solution, shift_data = shift_data_a, matrix = matrix_a, shuffle = shuffle)
   return resp

In [18]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

Cloning into 'Raianars_paper_One'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 277 (delta 83), reused 142 (delta 82), pack-reused 129
Receiving objects: 100% (277/277), 667.30 KiB | 10.76 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/Raianars_paper_One


In [19]:
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)

bounds=[]
for k in range(len(MAX)):
  bounds.append((MIN[k],MAX[k]))

In [20]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

Cloning into 'ANN_read_matrix'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), done.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/Raianars_paper_One/ANN_read_matrix


In [21]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')

In [22]:
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')

In [23]:
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)

In [24]:
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)

In [25]:
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')

In [26]:
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

In [27]:
##################################### 

TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

#dimensions=len(MAX)
dimensions=dim
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO


PAR=dim
Fun = Function
fobj=Fun

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
NPAR1=int(fator*NPAR);
NPAR2=int((NPAR-NPAR1)/3);
NPAR3=int((NPAR-NPAR1)/3);
NPAR4=int((NPAR-NPAR1)/3);
NPAR1 = NPAR-3*NPAR2
print('frac=',fator);
print('Npar1=',NPAR1);
print('Npar2=',NPAR2);
print('Npar3=',NPAR3);
print('Npar4=',NPAR4);
print('Npar=',NPAR)

cols=dim

X1=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X1[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
X2=np.zeros((NPAR2,cols)) # X2 is population DE (default DE)
for i in range(NPAR2):
  for j in range(cols):
    r=np.random.random()
    X2[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

X3=np.zeros((NPAR3,cols)) # X1 is Frannk population
for i in range(NPAR3):
  for j in range(cols):
    r=np.random.random()
    X3[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
X4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)
for i in range(NPAR4):
  for j in range(cols):
    r=np.random.random()
    X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
 ############################################################
  #bounds=[(5*dim, 15*dim)] * PONTOS # ITE
  #bounds=[(3*dim, 12*dim)] * PONTOS # ITE
  #bounds=[(4*dim, 16*dim)] * PONTOS
  
  # nao pode ser assim pq o bounds vale p o DE nao dos parametros
  #p=int(TOTAL/its/ITE) 
  #bounds=[(p, p)] * PONTOS


frac= 1.0
Npar1= 100
Npar2= 0
Npar3= 0
Npar4= 0
Npar= 100


In [28]:
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for iii in range(RUNS):

    FES = 0 ; gen = 0 
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)

    NPAR = maxNP
    # fator = 1.0
    NPAR1=int(fator*NPAR);
    '''
    NPAR2=int((NPAR-NPAR1)/3);
    NPAR3=int((NPAR-NPAR1)/3);
    NPAR4=int((NPAR-NPAR1)/3);
    NPAR1 = NPAR-3*NPAR2
    '''

    X1=np.zeros((NPAR1,cols)) # X1 is Frannk population
    for i in range(NPAR1):
      for j in range(cols):
        r=np.random.random()
        X1[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    '''    
    X2=np.zeros((NPAR2,cols)) # X2 is population DE (default DE)
    for i in range(NPAR2):
      for j in range(cols):
        r=np.random.random()
        X2[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

    X3=np.zeros((NPAR3,cols)) # X2 is population DE (default DE)
    for i in range(NPAR3):
      for j in range(cols):
        r=np.random.random()
        X3[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

    X4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)
    for i in range(NPAR4):
      for j in range(cols):
        r=np.random.random()
        X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    '''
    fit1 = np.asarray([fobj(ind) for ind in X1])
    best_idx = np.argmin(fit1)
    best = X1[best_idx]
    fbest = fit1[best_idx] 

    '''
    fit2 = np.asarray([fobj(ind) for ind in X2])
    fit3 = np.asarray([fobj(ind) for ind in X3])
    fit4 = np.asarray([fobj(ind) for ind in X4])
    FES = FES + NPAR1 + NPAR2 + NPAR3+ NPAR4
    '''
    FES = FES + NPAR1
    fit_best =[]
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    #Fitness = np.asarray([Fun(ind, shift_data , matrix, shuffle) for ind in X1])
    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X1,fit1 )
    nrow,ncol=X1.shape
    FES = FES + nrow

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo

    Fuma = Fun  # Fuma era Fun
  
    MAT_INDo=GeraIndices(X1,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fuma)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo

    # Just to intialize variables:
    FESr = 0
    FOBEST1 = 1E99;FOBEST2 = 1E99;FOBEST3 = 1E99;FOBEST4 = 1E99
    #BEST1,BEST2,BEST3,BEST4  =  np.copy(X1[0,:]),np.copy(X2[0,:]),np.copy(X3[0,:]), np.copy(X4[0,:]) 
    BEST1  =  np.copy(X1[0,:])
    popsize = NPAR
    FUNCTION = []
 
    MF=[]; MCR=[];
    for i in range(H):
      MCR.append(MCRi);MF.append(MFi) # line 3
 
    Narquive = popsize
    setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]
    while(FESr == 0 ):

      #print('FES',FES)

      gen = gen +1
      ##=cool=#####################################################################################
      '''
      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2),np.copy(X2),BEST2,FOBEST2,FES
      fit2,X2,BEST2,FOBEST2,XY2,BEST_XY2,FES=classicDE_FES_new.de(bounds, mut2, crossp2,NPAR2, its,fobj,fit2,X2,FES) # de_bounds_matriz
      if(FES > maxFES):
        FESr = FES 
        fit2,X2,BEST2,FOBEST2,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
      
      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit3),np.copy(X3),BEST3,FOBEST3,FES
      fit3,X3,BEST3,FOBEST3,XY3,BEST_XY3,FES=classicDE_FES_new.de(bounds, mut3, crossp3,NPAR3, its,fobj,fit3,X3,FES) #de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit3,X3,BEST3,FOBEST3,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit4),np.copy(X4),BEST4,FOBEST4,FES
      fit4,X4,BEST4,FOBEST4,XY4,BEST_XY4,FES=classicDE_FES_new.de(bounds, mut4, crossp4,NPAR4, its,fobj,fit4,X4,FES) # de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit4,X4,BEST4,FOBEST4,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
      '''
      
      

      

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit1),np.copy(X1),BEST1,FOBEST1,FES
      Xarqback = Xarq
      #fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES = annDE_FES_new.de(MAX,MIN, mut, crossp, NPAR1, its,fobj,fit1,X1,FES)
      fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade_new.LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
    
      if(FES > maxFES):
        FESr = FES 
        fit1,X1,BEST1,FOBEST1,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
        #Xarq = np.copy(Xarqback)
        Xarq = Xarqback
      calc = (minNP-maxNP)/maxFES*FES+maxNP # Linear Population Size Reduction (LPSR)
      calc = int(np.round(calc)) # Linear Population Size Reduction (LPSR) 
      gen = gen + 1
      #fator = 1.0
      NPAR = calc
      NPAR1=int(fator*NPAR);
      NPAR2=int((NPAR-NPAR1)/3);
      NPAR3=int((NPAR-NPAR1)/3);
      NPAR4=int((NPAR-NPAR1)/3);
      NPAR1 = NPAR-3*NPAR2
      '''
      if(NPAR2 > len(X2[:,0])):
        NPAR2 = len(X2[:,0])
      '''
      if(NPAR1 > len(X1[:,0])):
        NPAR1 = len(X1[:,0])

      
      X1 = X1[:NPAR1,:];
      #if(len(Xarq)>0) : Xarq = Xarq[:NPAR1,:]
      if(len(Xarq)>0) : Xarq = Xarq[:NPAR1]
      # X2 = X2[:NPAR2,:]; X3 = X3[:NPAR3,:]; X4 = X4[:NPAR4,:];
      fit1 = fit1[:NPAR1];# fit2 = fit2[:NPAR2]; fit3 = fit3[:NPAR3]; fit4 = fit4[:NPAR4];
      popsize = NPAR1
      Narquive = popsize
      NP = popsize
      '''
      X1[0,:] = BEST2;X1[1,:] = BEST3; X1[2,:] = BEST4;X1[3,:] = BEST1;
      X2[0,:] = BEST1;X2[1,:] = BEST3;X2[2,:] = BEST4;X2[3,:] = BEST2;
      X3[0,:] = BEST1;X3[1,:] = BEST2;X3[2,:] = BEST4;X3[3,:] = BEST3;
      X4[0,:] = BEST1;X4[1,:] = BEST2;X4[2,:] = BEST3;X4[3,:] = BEST4;
      fit1[0] = FOBEST1; fit2[1] = FOBEST2;fit3[0] = FOBEST3; fit4[1] = FOBEST4;
      '''
    '''
    for i in range(FIX):
      
        Fitback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2), np.copy(X2),BEST2,FOBEST2,FES
        
        if ( FESr == 0):
          Fitback,Xback,BESTback,FOBESTback,FESback= np.copy(X2),BEST2,FOBEST2,FES
          fit2,X2,BEST2,FOBEST2,XY2,BEST_XY2,FESr=classicDE_FES_new.de(bounds,mut_best, crossp_best,NPAR2, its,fobj,fit2,X2,FES) # de_bounds_matriz
        if ( FESr > 0):
          fit2, X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback


        if(FES > maxFES): 
          fit2, X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback
        
        
        fit2,X2,BEST2,FOBEST2,XY,BEST_XY, FES= stepsWALK_new.fixWALK(Fun,BEST2,FOBEST2,NPAR2,tunePAR,MAX,MIN,fit2,X2,FIX)
        # print('FES',FES)

      ############################################################################################
      ############################################################################################
    
    if(FES > maxFES):
      fit2,X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback

    FUNCTION.append(FOBEST2)
    '''
    #setTUNE= [SF,SCR,MF,MCR,p, terminal, Narquive]
    setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]
    print('run=',iii,'gen=',gen,'time=',(time.time()-start_time)/60, 'eval FES=', FES,'FO=',FOBEST1)  
    BEST_AGENTS.append(BEST_XY)

run= 0 gen= 12988 time= 2.605482546488444 eval FES= 299998 FO= 2.717528522781813e-90
run= 1 gen= 12988 time= 5.0274458765983585 eval FES= 299998 FO= 1.394958192103105e-85
run= 2 gen= 12988 time= 7.481686798731486 eval FES= 299998 FO= 2.095401567786058e-88
run= 3 gen= 12988 time= 9.936604619026184 eval FES= 299998 FO= 6.179590547577455e-88
run= 4 gen= 12988 time= 12.35445442199707 eval FES= 299998 FO= 2.523701013675979e-87
run= 5 gen= 12988 time= 14.767596157391866 eval FES= 299998 FO= 5.2169783465150784e-86
run= 6 gen= 12988 time= 17.172444971402488 eval FES= 299998 FO= 4.198803910695128e-89
run= 7 gen= 12988 time= 19.61718103090922 eval FES= 299998 FO= 1.8206906468786107e-89
run= 8 gen= 12988 time= 22.050901810328167 eval FES= 299998 FO= 1.5918317163906797e-88
run= 9 gen= 12988 time= 24.458215188980102 eval FES= 299998 FO= 5.294690751147574e-85
run= 10 gen= 12988 time= 26.860532820224762 eval FES= 299998 FO= 5.573811966332195e-85
run= 11 gen= 12988 time= 29.267805560429892 eval FES= 2

/content/optim_raianars_Insight/LShade_new.py:124: RuntimeWarning: invalid value encountered in double_scalars
  MCR[kH] = np.dot(w,y**2)/np.dot(w,y)


run= 14 gen= 12988 time= 36.619566237926485 eval FES= 299998 FO= 1.5166852209372343e-88
run= 15 gen= 12988 time= 39.052644010384874 eval FES= 299998 FO= 1.6687623470703272e-88
run= 16 gen= 12988 time= 41.46566475629807 eval FES= 299998 FO= 2.4321009667059843e-88
run= 17 gen= 12988 time= 43.87733388344447 eval FES= 299998 FO= 3.357261223838367e-85
run= 18 gen= 12988 time= 46.30371688206991 eval FES= 299998 FO= 1.1559445864908329e-87
run= 19 gen= 12988 time= 48.70437630017599 eval FES= 299998 FO= 6.0979731838433885e-87
run= 20 gen= 12988 time= 51.109295372168226 eval FES= 299998 FO= 7.887838662302573e-86
run= 21 gen= 12988 time= 53.51760319868723 eval FES= 299998 FO= 4.901643811541698e-88
run= 22 gen= 12988 time= 55.992575311660765 eval FES= 299998 FO= 1.8484479888411738e-85
run= 23 gen= 12988 time= 58.5234273036321 eval FES= 299998 FO= 5.498147726971109e-89
run= 24 gen= 12988 time= 60.9849989771843 eval FES= 299998 FO= 8.704458613542285e-88
run= 25 gen= 12988 time= 63.4049511273702 eval

In [29]:
#len(X2[1,:])
print(cols)

30


In [30]:
BEST_AGENTS=np.array(BEST_AGENTS)

In [31]:
NOMES=['DIVERSIDADE (DI)',  'DI RELATIVO','FRAC Its',
           'VELOC FOBJ','VELOC X','NP','Fo','CRo','fmin','fmax',
           'AD_fmin','AD_fmax','DELTA FOBJ','DELTA VX','r_fitness',
           'F','CR']
# BEST_INDEX= BEST_AGENTS[BEST_AGENTS[:,-1].argmin()]  
BEST_INDEX = np.argmin(BEST_AGENTS[:,-1])
#df=pd.DataFrame(MAT_stat[BEST_INDEX], columns=NOMES)



In [32]:
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()    

In [33]:
nome='SUB'+'_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('MCS-FRANNK','Fator F/DE= %.2f' %(fator),'NPAR=',NPAR,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1],'STD __:',STD)
print('MIN __:',MINFO, 'MAX__: ',MAXFO)
print(MEAN[-1])
print(STD)
print(MINFO)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)
# print('melhor=',BEST_AGENTS[BEST_INDEX])

MCS-FRANNK Fator F/DE= 1.00 NPAR= 4 Iterações= 1000 Dim= 30 Min/Max= -100 / 100 _ F1
SUB_Fo-0.5_CRo-0.5_D-30_Interv-1000 

MÉDIA:  8.761643462420441e-85 STD __: 5.243007872310012e-84
MIN __: 5.594046450029445e-91 MAX__:  3.748368724131455e-83
8.761643462420441e-85
5.243007872310012e-84
5.594046450029445e-91
Tempo Médio= 145.924 segundos ---
50 Rodadas

BEST:  [ 1.21995268e-46 -3.57317168e-48  2.10085363e-46 -2.12053441e-46
  4.43290148e-48 -4.88010980e-48 -6.35963793e-47 -5.12578597e-49
 -1.20143115e-48 -7.36831716e-49  3.20019278e-48 -8.74722995e-49
 -8.46648087e-48 -4.15731714e-49 -2.98247087e-50 -6.70881968e-49
  2.47121712e-48 -1.14547603e-49 -2.25515822e-48 -2.20273914e-50
  5.14989866e-50 -8.98012667e-52  2.43458140e-50 -6.88409162e-50
 -1.07733016e-48  1.06166574e-50 -4.14613231e-50  6.61327328e-51
  2.97692967e-51  7.39566312e-51  5.59404645e-91]


In [34]:
len(Xarq)

4

In [35]:
MCR

[0.9424514753126726,
 0.8011704504486683,
 0.8744903194880697,
 0.8611661784746254,
 0.8414635080152024,
 0.7876287985660003]

In [36]:
len(MF)

6